In [2]:
#%matplotlib widget #uncomment to be able rotate plot separately
# %matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets
from math import pi
from matplotlib import cm

# This function calculates a matrix, which transfroms given acceleration vector
def Dipolar_matrix(k, e_dd, theta, phi):
    if k==1:
        F1=0.8
        F2=0.8
    else:
        y1=np.power(1-k**2, 0.5, dtype=complex)
        F0=np.real(1/y1*np.log((1+y1)/(1-y1)))
        F1=1/4*(10/(1-k**2)-6/((1-k**2)**2)+3*(k**4)/((1-k**2)**2)*F0)
        F2=-1/(1-k**2)*(4-6/(1-k**2)+3/(1-k**2)*(k**2)*F0)
    ex=np.cos(phi)*np.sin(theta)
    ey=np.sin(phi)*np.sin(theta)
    ez=np.cos(theta)
    A11=1-e_dd+3*e_dd*(F1/4*(3*ex**2+ey**2)+(1-F1)*ez**2)
    A22=1-e_dd+3*e_dd*(F1/4*(ex**2+3*ey**2)+(1-F1)*ez**2)
    A33=1-e_dd+3*e_dd*(F2/4*(ex**2+ey**2)+(1-F2/2)*ez**2)
    A12=2*ex*ey*3*e_dd*F1/4
    A21=A12
    A13=2*ex*ez*3*e_dd*F2/4
    A23=2*ey*ez*3*e_dd*F2/4
    A31=2*ex*ez*3*e_dd*(1-F1)
    A32=2*ey*ez*3*e_dd*(1-F1)
    A=np.array([[A11,A12,A13],[A21,A22,A23],[A31,A32,A33]])
    B=np.linalg.inv(A)
    return B

#This function builds a general 3D plot of how effective acceleration is affected by dipolar interaction
def plot_3D_ellipsoid(k=0.35, e_dd=0.8, theta_B_deg=0.0, phi_B_deg=0.0,theta_A_deg=60):
    phi_B = np.radians(phi_B_deg) 
    theta_B = np.radians(theta_B_deg)
    phi_A = 0 
    theta_A = np.radians(theta_A_deg)
    
    f = plt.figure(figsize=(8, 4));
    ax=plt.axes(111,projection='3d');
   
    A_vec=-np.array([np.cos(phi_A)*np.sin(theta_A),np.sin(phi_A)*np.sin(theta_A),np.cos(theta_A)])
    B_vec=np.array([np.cos(phi_B)*np.sin(theta_B),np.sin(phi_B)*np.sin(theta_B),np.cos(theta_B)])
    C=Dipolar_matrix(k, e_dd, theta_B, phi_B)
    A_eff_vec=-C@A_vec

    N_p=150;
    phi_lin=np.linspace(0,2*pi,N_p)
    theta_lin=np.linspace(0,pi,N_p)
    X_surf=np.zeros((N_p,N_p))
    Y_surf=np.zeros((N_p,N_p))
    Z_surf=np.zeros((N_p,N_p))

    for i in range(len(phi_lin)):
        for j in range(len(theta_lin)):
            C1=Dipolar_matrix(k, e_dd, theta_lin[i], phi_lin[j])
            A0=-C1@A_vec
            X_surf[i,j]=A0[0]
            Y_surf[i,j]=A0[1]
            Z_surf[i,j]=A0[2]
    
    ax.plot_surface(X_surf,Y_surf,Z_surf, alpha=0.6, cmap=cm.Reds, rcount=50,
    ccount=50, shade=False, antialiased=True)

    ax.quiver(0,0,0,A_vec[0],A_vec[1],A_vec[2],color='red');
    ax.quiver(0,0,0,B_vec[0],B_vec[1],B_vec[2],color='purple');
    ax.quiver(0,0,0,A_eff_vec[0],A_eff_vec[1],A_eff_vec[2],color='green');
    ax.scatter(0,0,0,color='blue',s=8);
    ax.set_xlim(-1.5,1.5);
    ax.set_ylim(-1.5,1.5);
    ax.set_zlim(-1.5,1.5);
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    plt.show();
    # f.canvas.draw_idle()

# interact(plot_3D_ellipsoid,
#          k=(0.25,4,0.05),
#          e_dd=(0,1,0.1),
#          theta_B_deg=(0, 180, 2),
#          phi_B_deg=(0, 90, 2),
#          theta_A_deg=(0, 180, 2));

interactive(children=(FloatSlider(value=0.35, description='k', max=4.0, min=0.25, step=0.05), FloatSlider(valu…